In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [8]:
driver = webdriver.Chrome(service=Service("/home/zeki/dev_ws/EDA/driver/chromedriver-linux64/chromedriver"))#insert chromedriver file path
driver.get("https://jumin.mois.go.kr/ageStatMonth.do")
driver.set_window_position(2000, 0)
driver.maximize_window()
original_window=driver.current_window_handle

In [40]:
def start_year(year):
    
    start_year_panel=driver.find_element(By.XPATH,"""//*[@id="searchYearStart"]""")
    start_year_panel.send_keys(str(year))

In [41]:
def start_month(month):
    driver.find_element(By.XPATH,"""//*[@id="searchMonthStart"]/option[{}]""".format(13-month)).click()

In [45]:
def end_year(year):
    end_year_panel=driver.find_element(By.XPATH,"""//*[@id="searchYearEnd"]""")
    end_year_panel.send_keys(str(year))
def end_month(month):
    driver.find_element(By.XPATH,"""//*[@id="searchMonthEnd"]/option[{}]""".format(13-month)).click()

In [52]:
search_btn=driver.find_element(By.XPATH,"""//*[@id="content_main"]/div[2]/div[2]/div/div[1]/form/fieldset/div[2]/input[1]""")

In [53]:
csv_download=driver.find_element(By.XPATH,"""//*[@id="csvDown"]""")

In [72]:
driver.find_element(By.XPATH,"""//*[@id="gender"]""").click()

In [73]:
for year in range(2008,2025):
    start_year(year)
    end_year(year)
    
    for month in range(1,13):
        start_month(month)
        end_month(month)
        search_btn=driver.find_element(By.XPATH,"""//*[@id="content_main"]/div[2]/div[2]/div/div[1]/form/fieldset/div[2]/input[1]""")
        driver.implicitly_wait(10)
        EC.element_to_be_clickable(search_btn.click())
        driver.implicitly_wait(10)
        csv_download=driver.find_element(By.XPATH,"""//*[@id="csvDown"]""")
        csv_download.click()
        try:
            result = driver.switch_to.alert
            result.accept()
        except:
            pass

In [135]:
driver.quit()

In [74]:
import pandas as pd

In [297]:
start_date = '012008'
end_date = '072024'

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
date_range_str = date_range.strftime('%Y%m')

df_list=[]
for tmp_date in date_range_str:
    monthly_data=pd.read_csv("../data/population_monthly_data/{}_{}_연령별인구현황_월간.csv".format(tmp_date,tmp_date),encoding='EUC-KR')
    df=pd.DataFrame(monthly_data)

    df.drop(columns=df.columns[2], axis=1, inplace=True)
    df.rename(columns= { df.columns[0] : "city",df.columns[1] : "all_popul",
                        df.columns[2] : "age_0_9",
                        df.columns[3] : "age_10_19",
                        df.columns[4] : "age_20_29",
                        df.columns[5] : "age_30_39",
                        df.columns[6] : "age_40_49",
                        df.columns[7] : "age_50_59",
                        df.columns[8] : "age_60_69",
                        df.columns[9] : "age_70_79",
                        df.columns[10] : "age_80_89",
                        df.columns[11] : "age_90_99",
                        df.columns[12] : "age_100+"


                        }, inplace=True)
    data_year=tmp_date[:4]
    data_month=tmp_date[4:]
                        
    df.insert(1,'month',data_month)
    df.insert(1,'year',data_year)

    df['city']=df['city'].str.replace(r'\(.*\)','',regex=True)


    df_list.append(df)
    
all_df=pd.concat(df_list)

for column in range(3,len(all_df.columns)):
    all_df[all_df.columns[column]]=all_df[all_df.columns[column]].str.replace(',','')

    all_df =all_df.astype(dtype={all_df.columns[column]:'int'},
                              errors='raise',
                              copy=True)
all_df[all_df.columns[0]]=all_df[all_df.columns[0]].str.replace(' ','')
all_df[all_df.columns[0]]=all_df[all_df.columns[0]].str.replace('강원특별자치도','강원도')
all_df[all_df.columns[0]]=all_df[all_df.columns[0]].str.replace('전북특별자치도','전라북도')


/tmp/ipykernel_8357/123484654.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq='M')


In [298]:
all_df.iloc[0][2]

/tmp/ipykernel_8357/1277428550.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all_df.iloc[0][2]


'01'

In [299]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3510 entries, 0 to 17
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   city       3510 non-null   object
 1   year       3510 non-null   object
 2   month      3510 non-null   object
 3   all_popul  3510 non-null   int64 
 4   age_0_9    3510 non-null   int64 
 5   age_10_19  3510 non-null   int64 
 6   age_20_29  3510 non-null   int64 
 7   age_30_39  3510 non-null   int64 
 8   age_40_49  3510 non-null   int64 
 9   age_50_59  3510 non-null   int64 
 10  age_60_69  3510 non-null   int64 
 11  age_70_79  3510 non-null   int64 
 12  age_80_89  3510 non-null   int64 
 13  age_90_99  3510 non-null   int64 
 14  age_100+   3510 non-null   int64 
dtypes: int64(12), object(3)
memory usage: 438.8+ KB


In [300]:
all_df.reset_index(drop=True,inplace=True)


<<<데이터프레임 생성완료/db업로드

In [301]:
import mysql.connector
remote = mysql.connector.connect(
    host = "database-1.************.amazonaws.com", #insert sql path
    port = 3306,
    user = "eda",
    password = "*******",
    database= 'eda'
)
cur = remote.cursor(buffered=True)


In [285]:
remote.close()

In [302]:
sql="""create table population(

            city varchar(32),
            year char(4), 
            month char(2),
            pop_sum int,
            pop_0_9 int,
            pop_10_19 int,
            pop_20_29 int,
            pop_30_39 int,
            pop_40_49 int,
            pop_50_59 int,
            pop_60_69 int,
            pop_70_79 int,
            pop_80_89 int,
            pop_90_99 int,
            pop_100_ int

            );"""
cur.execute(sql)
remote.commit()

In [303]:
sql="""insert into population Values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""
for i , row in all_df.iterrows():
    cur.execute(sql,tuple(row))
    # print(tuple(row))
    remote.commit()

In [304]:
remote.close()

///끝